In [165]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import keyboard

In [166]:
mp_hands = mp.solutions.hands
hands=mp_hands.Hands(max_num_hands=1,min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils


In [167]:
# Initialize the video capture
cap = cv2.VideoCapture(0)


In [168]:
def state (hand_landmarks,frame):
    # Detecting hand state

                wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]


                index_finger_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
                
                
                # f.write(str(index_finger_tip.x) + "," + str(wrist.x)+ "," + str(index_finger_tip.x-wrist.x)  + "\n")

                for_back=0
                left_right=0
                #open are close
                if (index_finger_mcp.y>index_finger_tip.y):
                    for_back=1
                    
                else:
                    for_back=2

                # #left are right
                if ((index_finger_tip.x-wrist.x)<-.20):
                    left_right=3    
                elif ((index_finger_tip.x-wrist.x)>.05):
                    left_right=4
                

                return for_back,left_right

In [169]:
def is_state_changed(new_for_back,new_left_right,old_for_back,old_left_right):
    direction=[' ','w','s','a','d']
    if old_for_back==old_left_right==0:
        keyboard.press(direction[new_for_back])
        if  old_left_right != 0:
            keyboard.press(direction[old_left_right])
        return False
    elif new_for_back==old_for_back and new_left_right==old_left_right:
        return False
    else: return True   
    

In [170]:
#opening excel sheet
from ast import And
from calendar import c
from hmac import new
from math import pi
direction=[' ' ,'w','s','a','d']
old_for_back=0
old_left_right=0
f = open("coordinaates.csv", "w")
while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Hands
        results = hands.process(rgb_frame)
        if results.multi_hand_landmarks:                     #it means that the MediaPipe Hands model has detected one or more hands in the frame
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )
                
                
                new_for_back,new_left_right=state(hand_landmarks,frame)
                if is_state_changed( new_for_back,new_left_right,old_for_back,old_left_right):
                    if old_for_back!=0 and old_left_right!=0:
                        keyboard.release(direction[old_for_back],direction[old_left_right])
                    elif old_for_back!=0:
                        keyboard.release(direction[old_for_back])
                    elif old_left_right!=0:
                        keyboard.release(direction[old_left_right])
                    
                    if new_for_back!=0 and new_left_right!=0:
                        keyboard.press(direction[new_for_back],direction[new_left_right])
                    elif new_for_back!=0:
                        keyboard.press(direction[new_for_back])
                    elif new_left_right!=0:
                        keyboard.press(direction[new_left_right])
                            


                        #
                    old_for_back=new_for_back
                    old_left_right=new_left_right
                # cv2.putText(frame, str(for_back), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                # cv2.putText(frame, str(left_right), (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                
        # Display the frame
        cv2.imshow('MediaPipe Hand Gesture', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

d:\study\codes\engineering_codes\research\conda_env_for_project\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [171]:
cap.release()
f.close()
cv2.destroyAllWindows()